<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_opsmps3copy.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# GetWork App  📒
***Create your own Tapis App***
THIS WORKS, KIND OF, BUT IT'S SO CONVOLUTED AND STILL NOT CLEAR.
by Silvia Mazzoni, DesignSafe, 2025

Let's write an app to get your user and system-dependent path

## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
import json

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [3]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-20T05:35:08+00:00
 Token expires in: 1:32:07.081229
-- LOG IN SUCCESSFUL! --


---
### Get username

In [4]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [5]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

username silvia


## Define actions!

In [6]:
# Make these True once you are done validating
do_makeApp = True
do_submitJob = True

---
## Configure App

In [7]:
app_id = 'get-work-path-test'
app_description = 'Run any Executable'

### AutoIncrement app_version

In [8]:
if do_makeApp:
    latest_app_version = OpsUtils.get_latest_app_version(t,app_id)
    print('now latest_app_version',latest_app_version)
    app_version = OpsUtils.bump_app_version(latest_app_version,'patch')
    print('now app_version',app_version)

now latest_app_version 0.0.3
now app_version 0.0.4


### Where to store the files

In [9]:
if do_makeApp:
    app_system_id="designsafe.storage.default"
    app_path = f"{username}/apps/{app_id}/{app_version}"
    container_filename = f'{app_id}.zip'

### Define folder structure: app_id/version

In [10]:
if do_makeApp:
    app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [11]:
if do_makeApp:
    app_folder = os.path.abspath(os.path.expanduser(app_folderName))
    os.makedirs(app_folder, exist_ok=True)
    print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

app_folder: /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.4
 exists: True


---
## Create the App Files

A Tapis app needs the following **core files**:

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [12]:
# let's collect the list of files we are creating as we will need this list in upload
if do_makeApp:
    appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [13]:
if do_makeApp:
    thisFilename = 'ReadMe.MD'
    thisText = """\
    # making a dummy app to get user-and-system-specific work path
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [14]:
if do_makeApp:
    thisFilename = 'app.json'
    thisText = """\
    {
      "id": "__app_id__",
      "version": "__app_version__",
      "description": "__app_description__",
      "owner": "${apiUserId}",
      "enabled": true,
      "runtime": "ZIP",
      "runtimeVersion": null,
      "runtimeOptions": null,
      "containerImage": "__container_filename_path__",
      "jobType": "BATCH",
      "maxJobs": -1,
      "maxJobsPerUser": -1,
      "strictFileInputs": true,
      "jobAttributes": {
        "execSystemConstraints": null,
        "execSystemId": "stampede3",
        "execSystemExecDir": "${JobWorkingDir}",
        "execSystemInputDir": "${JobWorkingDir}",
        "execSystemOutputDir": "${JobWorkingDir}",
        "execSystemLogicalQueue": "skx",
        "archiveSystemId": "stampede3",
        "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
        "archiveOnAppError": true,
        "isMpi": false,
        "mpiCmd": null,
        "parameterSet": {
          "appArgs": [],
          "containerArgs": [],
          "schedulerOptions": [
            {
              "name": "OpenSees TACC Scheduler Profile",
              "description": "Scheduler profile for the default version of OpenSees",
              "inputMode": "FIXED",
              "arg": "--tapis-profile OpenSees_default",
              "notes": {
                "isHidden": true
              }
            }
          ],
          "envVariables": [],
          "archiveFilter": {
            "includes": [],
            "excludes": ["__container_filename__"],
            "includeLaunchFiles": true
          }
        },
        "fileInputs": [],
        "fileInputArrays": [],
        "nodeCount": 1,
        "coresPerNode": 1,
        "memoryMB": 192000,
        "maxMinutes": 120,
        "subscriptions": [],
        "tags": []
      },
      "tags": [
        "portalName: DesignSafe",
        "portalName: CEP"
      ],
      "notes": {
        "label": "__app_id__",
        "helpUrl": "",
        "hideNodeCountAndCoresPerNode": false,
        "isInteractive": false,
        "icon": "OpenSees",
        "category": "Simulation"
      }
    }
    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    thisText = thisText.replace("__app_description__", app_description)
    thisText = thisText.replace("__container_filename_path__", f"tapis://{app_system_id}/{app_path}/{container_filename}")
    thisText = thisText.replace("__container_filename__", container_filename)
  
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [15]:
if do_makeApp:
    thisFilename = 'profile.json'
    thisText = """\
    {
        "name": "__app_id__",
        "description": "Modules to load",
        "moduleLoads": [ ],
        "hiddenOptions": [
            "MEM"
        ]
    }
    """ 
    thisText = thisText.replace("__app_id__", app_id)
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [16]:
if do_makeApp:
    import textwrap, time
    from zipfile import ZipFile, ZIP_DEFLATED, ZipInfo
    
    thisFilename_sh = "tapisjob_app.sh"
    thisFilename = container_filename  # existing var


    bash_script = textwrap.dedent("""\
        #!/bin/bash
        set -euo pipefail
        set -x

        inputDirectory="${inputDirectory:?inputDirectory not set}"
        echo "$inputDirectory"
    """)


    zip_path = os.path.join(app_folder, thisFilename)
    print('zip_path',zip_path)

    # write with executable permission inside the ZIP
    zi = ZipInfo(thisFilename_sh)
    zi.date_time = time.localtime(time.time())[:6]
    zi.compress_type = ZIP_DEFLATED
    zi.external_attr = 0o100755 << 16   # -rwxr-xr-x on a regular file
    
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
        z.writestr(zi, bash_script)
        
    appFilesList.append(thisFilename)

zip_path /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.4/get-work-path-test.zip


#### File Check
Look at the files we have written and check for typos or formatting errors.

In [17]:
if do_makeApp:
    print(appFilesList)
    OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

['ReadMe.MD', 'app.json', 'profile.json', 'get-work-path-test.zip']


---
## Validate App Locally

In [18]:
if do_makeApp:
    OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [19]:
if do_makeApp:
    validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
    if not validation:
        print('Validation Failed: stopping here!!!!')
        a = 3/0

🔍 Validating app folder: /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.4

✅ All required files are present.

📄 App ID: get-work-path-test
📄 App Name: get-work-path-test
📄 Version: 0.0.4
🔧 Parameters: []
📦 Inputs: []
📤 Outputs: []

App Keys: ['id', 'version', 'description', 'owner', 'enabled', 'runtime', 'runtimeVersion', 'runtimeOptions', 'containerImage', 'jobType', 'maxJobs', 'maxJobsPerUser', 'strictFileInputs', 'jobAttributes', 'tags', 'notes', 'name', 'moduleLoads', 'hiddenOptions']

✅ Basic validation complete. App folder looks good!


---
## Deploy the App

### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [20]:
if do_makeApp:
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

app_path silvia/apps/get-work-path-test/0.0.4


### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [21]:
if do_makeApp:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [22]:
if do_makeApp:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    appfiles = t.files.listFiles(systemId=app_system_id, path=app_path)
    for thisF in appfiles:
        print(thisF)
        print('')

app_system_id: designsafe.storage.default
app_path: silvia/apps/get-work-path-test/0.0.4

group: 819066
lastModified: 2025-08-20T04:03:02Z
mimeType: application/json
name: app.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.4/app.json
size: 2205
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-path-test/0.0.4/app.json


group: 819066
lastModified: 2025-08-20T04:03:02Z
mimeType: application/zip
name: get-work-path-test.zip
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.4/get-work-path-test.zip
size: 208
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-path-test/0.0.4/get-work-path-test.zip


group: 819066
lastModified: 2025-08-20T04:03:02Z
mimeType: application/json
name: profile.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.4/profile.json
size: 179
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-pa

---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [23]:
if do_makeApp:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List all apps

In [24]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

--

created: 2024-12-13T20:15:33.014747Z
description: Run shell commands on remote systems
id: shell-runner-1.0.0
owner: silvia
version: 1.0.0
--

created: 2024-12-14T15:49:21.510206Z
description: Run shell commands on remote systems
id: shell-runner
owner: silvia
version: 1.0.0
--

created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
owner: silvia
version: latest
--

created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
owner: silvia
version: latest
--

created: 2025-08-17T22:34:01.193589Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-copy-silvia
owner: silvia
vers

#### List the new app

In [25]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
print(appMetaData)


containerImage: tapis://designsafe.storage.default/silvia/apps/get-work-path-test/0.0.4/get-work-path-test.zip
created: 2025-08-20T04:03:03.366218Z
deleted: False
description: Run any Executable
enabled: True
id: get-work-path-test
isPublic: False
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: stampede3
cmdPrefix: None
coresPerNode: 1
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: stampede3
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: skx
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: []
isMpi: False
maxMinutes: 120
memoryMB: 192000
mpiCmd: None
nodeCount: 1
parameterSet: 
appArgs: []
archiveFilter: 
excludes: ['get-work-path-test.zip']
includeLaunchFiles: True
includes: []
containerArgs: []
envVariables:

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [26]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [27]:
tapisInputAll["maxMinutes"] = 1

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [28]:
print('app_id:',app_id)

app_id: get-work-path-test


### OpenSees

In [29]:


# tapisInput['storage_system'] = 'CommunityData'
# tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 


# # -----------------------------------------------------
# if do_submitJob:
#     here_tapisInput = tapisInput.copy()
#     here_tapisInput["appId"] = app_id 
#     here_tapisInput["name"] = here_tapisInput["appId"]
#     print(f'\n -- {here_tapisInput['name']} --\n')
#     jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# # -----------------------------------------------------

In [33]:
   
from tapipy.tapis import TapisResult
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
app_MetaData = appMetaData.__dict__
print(app_MetaData.keys())
app_jobAttributes = app_MetaData['jobAttributes'].__dict__
print('app_jobAttributes',app_jobAttributes.keys())
app_parameterSet = app_jobAttributes['parameterSet'].__dict__
print('app_parameterSet',app_parameterSet.keys())
# app_appArgs = app_parameterSet['appArgs']
# app_envVariables = app_parameterSet['envVariables']
for app_key in ['appArgs','envVariables']:
    print('app_key',app_key)
    app_Dict = app_parameterSet[app_key]
    print(app_Dict)
for app_key in ['fileInputs']:
    print('app_key',app_key)
    app_Dict = app_jobAttributes[app_key]
    print(app_Dict)
    

job_description = {'name':'getWork'}
job_description["appId"] = app_id
job_description["appVersion"] = app_version
here_tapisInput = tapisInputAll.copy()
here_tapisInput['storage_system'] = 'CommunityData'
here_tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 


dict_keys(['sharedAppCtx', 'isPublic', 'sharedWithUsers', 'tenant', 'id', 'version', 'description', 'owner', 'enabled', 'versionEnabled', 'locked', 'runtime', 'runtimeVersion', 'runtimeOptions', 'containerImage', 'jobType', 'maxJobs', 'maxJobsPerUser', 'strictFileInputs', 'jobAttributes', 'tags', 'notes', 'uuid', 'deleted', 'created', 'updated'])
app_jobAttributes dict_keys(['description', 'dynamicExecSystem', 'execSystemConstraints', 'execSystemId', 'execSystemExecDir', 'execSystemInputDir', 'execSystemOutputDir', 'dtnSystemInputDir', 'dtnSystemOutputDir', 'execSystemLogicalQueue', 'archiveSystemId', 'archiveSystemDir', 'archiveOnAppError', 'isMpi', 'mpiCmd', 'cmdPrefix', 'parameterSet', 'fileInputs', 'fileInputArrays', 'nodeCount', 'coresPerNode', 'memoryMB', 'maxMinutes', 'subscriptions', 'tags'])
app_parameterSet dict_keys(['appArgs', 'containerArgs', 'schedulerOptions', 'envVariables', 'archiveFilter', 'logConfig'])
app_key appArgs
[]
app_key envVariables
[]
app_key fileInputs
[]


In [34]:
jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,job_description=job_description,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)

Submitting Job
Job submitted! ID: 177c6dc6-14c4-406b-aeb1-4263a09c3ce6-007
job_start_time: 1755662715.730435

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.05 sec	 Current Status: PENDING
	 Elapsed job time: 2.08 sec	 Current Status: STAGING_JOB		(PENDING took 1.03 sec)
	 Elapsed job time: 23.86 sec	 Current Status: SUBMITTING_JOB		(STAGING_JOB took 21.78 sec)
	 Elapsed job time: 24.9 sec	 Current Status: FAILED		(SUBMITTING_JOB took 1.03 sec)
	  Status: FAILED	 Elapsed job time: 24.9 sec
--------------------
Elapsed time since Job was submitted: 24.9 sec
--------------------


Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (177c6dc6-14c4-406b-aeb1-4263a09c3ce6-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (177c6dc6-14c4-406b-aeb1-4263a09c3ce…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (177c6dc6-14c4-406b-aeb1-4263a09…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (177c6dc6-14c4-406b-aeb1-4263a09c3ce…